How to use the Twitter API and Py-ART to Tweet Your Australian Radar!
----------------------------------------------------------

*Scott Collis, Argonne National Laboratory* Adapted for Australian radars by Joshua Soderholm

In [1]:
#First some imports
import twitter #https://python-twitter.readthedocs.io/en/latest/index.html
import json
import pyart #https://github.com/ARM-DOE/pyart
from mpl_toolkits.basemap import Basemap
from owslib.wms import WebMapService
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from boto.s3.connection import S3Connection #Anaconda installable
import shutil, os
from datetime import timedelta, datetime
import numpy as np
import tempfile
import shutil, os


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119 


/home/meso/anaconda2/lib/python2.7/site-packages/ipykernel/kernelbase.py:43: DeprecationWarning: IPythonKernel._eventloop_changed is deprecated: use @observe and @unobserve instead.
  def _eventloop_changed(self, name, old, new):


First, some functions to deal with Amazon Web Services Simple Storage Service (s3) holding of BOM radar archive. This is a realtime and historical archive provided by Fugro-ROAMES for testing purposes. Future aim is to host this archive permanently, publically.

In [2]:
def nearestDate(dates, pivot):
    return min(dates, key=lambda x: abs(x - pivot))


def get_radar_from_aws(site, datetime_t):
    """
    Get the closest volume of NEXRAD data to a particular datetime.
    Parameters
    ----------
    site : string
        four letter radar designation 
    datetime_t : datetime
        desired date time
    """
    
    #First create the query string for the bucket knowing
    #how NOAA and AWS store the data
    
    my_pref = "odimh5_archive/" + site + datetime_t.strftime('/%Y/%m/%d')
    #Connect to the bucket
    
    conn = S3Connection(anon = True)
    bucket = conn.get_bucket('roames-wxradar-archive')
    
    #Get a list of files 
    
    bucket_list = list(bucket.list(prefix = my_pref))
    #print(bucket_list)
    #we are going to create a list of keys and datetimes to allow easy searching
    
    keys = []
    datetimes = []
    
    #populate the list
    for i in range(len(bucket_list)):
        this_str = str(bucket_list[i].key)
        if 'h5' in this_str:
            endme = this_str[-18:-3]
            fmt = '%Y%m%d_%H%M%S' 
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])
            #print(dt)
    
    #find the closest available radar to your datetime 
    closest_datetime = nearestDate(datetimes, datetime_t)
    index = datetimes.index(closest_datetime)
    #print(closest_datetime)
    #create a temp file, download radar data to file from S3
    #read into a radar object and return
    
    localfile = tempfile.NamedTemporaryFile()
    keys[index].get_contents_to_filename(localfile.name)
    radar = pyart.aux_io.read_odim_h5(localfile.name, file_field_names=True) 
    return radar

Ok.. now the part that takes some work.. Take a loom here:

https://python-twitter.readthedocs.io/en/latest/getting_started.html

This gives you instruction on how to populate the json file... 

Here is an example:
`
➜  twitterradar git:(tools_init) ✗ more token/PyWeather.json 
{"consumer_key":"SOMETHING",
"consumer_secret":"SOMETHINGELSE",
"access_token_key":"ANOTHERTHING",
"access_token_secret":"YETANOTHERTHING"}
`



In [8]:
fh = open('/home/meso/aws_key/twitter_key.json')
myson = json.load(fh)
fh.close()

Initialize the API

In [9]:
api = twitter.Api(consumer_key=myson['consumer_key'],
                  consumer_secret=myson['consumer_secret'],
                  access_token_key=myson['access_token_key'],
                  access_token_secret=myson['access_token_secret'])

Ok.. Now a cool method for tweeting your radar!

In [10]:
def tweet_my_radar(datetime, radar, tapi, min_lat = None,
                  max_lat = None, min_lon = None, 
                  max_lon = None, fig_sz = None):
    """
    Fetch a radar from S3, plot it and tweet plus statistics.
    
    Grab a radar from a site and use the Twitter API
    to tweet the PPI from the lowest tilt to twitter.
    Also tweet the number of gates above two reflectivity
    thresholds and the min and max reflectivity.
    
    Parameters
    ----------
    datetime: datetime object
        Python datetime object to be passed to the method
        to find the nearest radar object from AWS S3 using
        boto. 
    
    radar: String
        Two number radar code (e.g., 02 Sydney, 66 Brisbane)
    
    min_lat, max_lat, min_lon, max_lon: floats
        bounds for the display
        
    Returns
    -------
    None
    
    """
    fig = plt.figure(figsize = fig_sz)
    #setup basemap
    ref_m = Basemap(llcrnrlon=min_lon,
                llcrnrlat=min_lat,
                urcrnrlon=max_lon,
                urcrnrlat=max_lat, 
                projection='tmerc', 
                resolution = 'f',
                epsg = 3857)
    
    #load background image
    im = plt.imread('background.png')
    ref_m.imshow(im,zorder = 0,origin='upper')
    
    my_radar = get_radar_from_aws(radar,b_d )
    my_radar.fields['DBZH']['standard_name'] = 'Reflectivity'
    my_radar.fields['DBZH']['units'] = 'dBZ'
    my_radar.fields['DBZH']['long_name'] = 'Radar Reflectivity Factor'
    #Make a display
    display = pyart.graph.RadarMapDisplay(my_radar)
    #Plot Z from lowest tilt
    display.plot_ppi_map('DBZH', sweep = 1, resolution = 'f',
                        vmin = -8, vmax = 64, mask_outside = True,
                        cmap = pyart.graph.cm.NWSRef,basemap = ref_m,zorder = 1)

    #overlay mapping data
    im = plt.imread('overlay.png')
    ref_m.imshow(im,zorder = 2,origin='upper')
    #get a tempfile
    localfile = tempfile.NamedTemporaryFile()
    #Save to tempfile.. Need png or Twitter gets grumpy 
    plt.savefig(localfile.name + '.png', dpi = 200)
    #Now grab some statistics.. 
    min_z = my_radar.fields['DBZH']['data'].min()
    max_z = my_radar.fields['DBZH']['data'].max()
    n_gates_20 = len(np.where(my_radar.fields['DBZH']['data'] > 20.0)[0])
    n_gates_40 = len(np.where(my_radar.fields['DBZH']['data'] > 40.0)[0])
    
    #Format the strings.. TODO: turn into percent
    gdata = "There are {0} gates above 20dBZ and {1} above 40dBZ".format(n_gates_20, 
                                                                     n_gates_40)
    
    #Make the tweet text
    mmdata = "The min Z is {0}dBZ and the max is {1}dBZ".format(min_z,
                                                            max_z)
    #And... post it.. yes.. it is that easy!
    tapi.PostUpdate( gdata + ' ' + mmdata, 
              media = localfile.name + '.png')
    

In [11]:
def generate_layers(max_lat,min_lat,max_lon,min_lon):
    
    #generate map bounds
    lat_dif   = max_lat-min_lat
    lon_dif   = max_lon-min_lon
    map_x_sz  = int(500*lon_dif)
    map_y_sz  = int(500*lat_dif)

    #create overly map
    wms = WebMapService('http://services.ga.gov.au/site_7/services/Topographic_Base_Map_WM/MapServer/WMSServer?', version='1.1.1')
    img = wms.getmap(layers=['Roads_4','Populated_Places_6'],srs='EPSG:4326',bbox=(min_lon, min_lat, max_lon, max_lat),size=(map_x_sz, map_y_sz),format='image/png',transparent=True)
    out = open('overlay.png', 'wb')
    out.write(img.read())
    out.close()  

    #create background map
    wms = WebMapService('http://ows.terrestris.de/osm-gray/service?', version='1.1.1')
    img = wms.getmap(layers=['TOPO-WMS'],srs='EPSG:4326',bbox=(min_lon, min_lat, max_lon, max_lat),size=(map_x_sz, map_y_sz),format='image/png',transparent=True)
    out = open('background.png', 'wb')
    out.write(img.read())
    out.close() 

In [12]:
#2014 hailstorm for Brisbane

fig_sz    = [7,8]
base_date = "20141127_060000"
radar     = "66"
fmt       = '%Y%m%d_%H%M%S' 
b_d       = datetime.strptime(base_date, fmt)
min_lat   = -28.6
max_lat   = -26.8
min_lon   = 152.7
max_lon   = 153.7  

generate_layers(min_lat = min_lat,max_lat = max_lat,
                min_lon = min_lon, max_lon = max_lon)

#process img
tweet_my_radar(b_d, radar, api, min_lat = min_lat,
                  max_lat = max_lat, min_lon = min_lon, 
                  max_lon = max_lon, fig_sz = fig_sz)

TwitterError: [{u'message': u'Bad Authentication data.', u'code': 215}]

In [180]:
#now for Brisbane

fig_sz    = [7,8]
radar   = "66"
b_d     = datetime.now()
min_lat = -28.6
max_lat = -26.8
min_lon = 152.7
max_lon = 153.7

generate_layers(min_lat = min_lat,max_lat = max_lat,
                min_lon = min_lon, max_lon = max_lon,)

#process img
tweet_my_radar(b_d, radar, api, min_lat = min_lat,
                  max_lat = max_lat, min_lon = min_lon, 
                  max_lon = max_lon, fig_sz = fig_sz)

http://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/export?bbox=3648270.83445,-3681909.36342,3826224.46982,-3496818.08908&bboxSR=3825&imageSR=3825&size=750,780&dpi=96&format=png32&f=image
